In [22]:
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib as plt
import seaborn as sns
import os
%matplotlib inline

In [32]:
df = pd.DataFrame()
for filename in os.listdir("stocks/"):
    path = "stocks/" + filename
    col1 = filename.split(".")[0] + 'close'
    col2 = filename.split(".")[0] + 'volume'
    series[filename] = pd.read_csv(path, parse_dates=True, sep=',', header=0, index_col=0, 
                                   usecols=['date', col1, col2])
    print series[filename].head()

ValueError: Usecols do not match names.